In [1]:
pip install ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from ollama_interface import chat_with_model
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

In [4]:
# === CONFIG ===
PROMPT_DIR = "./prompt_templates"
PROMPT_FILES = ["PS1.txt", "PS2.txt", "PS3.txt", "PS4.txt", "PS5.txt"]

In [5]:
TOPICS = [
    "Decision Tree Models",
    "Training, Validation, and Testing of Machine Learning Models",
    "Gradient Boosted Tree Models",
    "Linear Regression, Logistic Regression, and Multilayer Perceptron",
    "Stochastic Gradient Descent",
    "Backpropagation",
    "Foundations of Computer Vision and Convolutional Neural Networks",
    "Transfer Learning for Computer Vision",
    "Image Segmentation and Object Detection",
    "Data Pre-processing for Natural Language Processing Tasks",
    "Bag of Words Approach and Word Embedding",
    "Attention Mechanism in Transformers",
    "Neural Machine Translation using Transformers",
    "Encoder, Decoder, and Sequence-to-Sequence Transformers",
    "Pretraining, Finetuning, and Reinforcement Learning with Human Feedback",
    "Prompt Engineering and Chain of Thought Prompting",
    "Natural Language Processing Tasks and Transformer Architectures Used for the Tasks"
]

In [ ]:
default_options = {
    "temperature": 0.8,
    "top_p": 0.9,
    "top_k": 40,
    "num_predict": -1,
    "presence_penalty": 0.0,
    "frequency_penalty": 0.0,
    "repeat_penalty": 1.1,
    "stop": [],
    "seed": None,
    "mirostat": 0,
    "mirostat_tau": 5.0,
    "mirostat_eta": 0.1,
    "grammar": None
}

SUPPORTED_MODELS = ["gpt-oss:latest", "granite4:latest", "mistral-small3.2:latest"] # Models used

In [7]:
def fill_prompt_template(template_text, values_dict):
    for key, value in values_dict.items():
        template_text = template_text.replace(f"{{{key}}}", value)
    return template_text

def generate_questions_for_topic(topic):
    return {
        "topic": topic,
        "Remember_question": f"Define {topic}?",
        "Understand_question": f"Can you explain the key ideas behind {topic}?",
        "Apply_question": f"How would you apply {topic} in a real-world scenario?",
        "Analyze_question": f"Analyze the components and working of {topic}.",
        "Evaluate_question": f"Evaluate the strengths and limitations of {topic}.",
        "Create_question": f"Design an experiment or application using {topic}."
    }

In [8]:
wb = Workbook()
ws = wb.active
ws.title = "Model Responses"

# Set headers
ws.append(["Index", "Prompt", "Model", "Responses"])

In [ ]:
row_index = 2
index = 1

for prompt_file in PROMPT_FILES:
    with open(os.path.join(PROMPT_DIR, prompt_file), "r", encoding="utf-8") as f:
        template_text = f.read()

    for topic in TOPICS:
        input_data = generate_questions_for_topic(topic)
        filled_prompt = fill_prompt_template(template_text, input_data)

        # First row for this topic gets the prompt text
        first_entry = True
        start_row = row_index

        for model in SUPPORTED_MODELS:
            print(f"🧠 Running: {prompt_file} | Topic: {topic} | Model: {model}")
            messages = [
                {"role": "system", "content": "You are a helpful assistant."}, # For Future Experiments include output formatting (eg, JSON) for easy parsing
                {"role": "user", "content": filled_prompt}
            ]

            try:
                response = chat_with_model(
                    model_name=model,
                    messages=messages,
                    options=default_options
                )
                answer = response["message"]["content"].strip()
                print(answer)
            except Exception as e:
                answer = f"Error: {e}"

            ws.append([index, filled_prompt if first_entry else "", model, answer])
            index += 1
            row_index += 1
            first_entry = False

🧠 Running: PS1.txt | Topic: Decision Tree Models | Model: gpt-oss:latest
**Decision‑Tree‑Model Questions (Revised Bloom’s Taxonomy)**  

| Cognitive Level | Question (Indian context) | What it tests |
|------------------|---------------------------|---------------|
| **Remember** | 1. *Define a decision‑tree model in the context of machine learning and explain the typical terminology used in India‑based educational data (e.g., “class”, “node”, “leaf”, “feature”).* | Recall of basic definition and key terms. |
| **Understand** | 2. *Explain how entropy (or Gini impurity) guides the selection of the first attribute to split on in a decision tree. Illustrate with an example using an Indian dataset of students’ “Score”, “Attendance”, and “Pass/Fail”.* | Demonstrates comprehension of the concept and its application. |
| **Apply** | 3. *Using the ID3 algorithm, construct a decision tree for the following mini‑dataset of 8 Indian students:  <br>```
| Study Hours | Attendance | Pass/Fail |
|--

In [ ]:
# Optional: auto-size columns
for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max(15, min(max_length + 2, 50))

In [ ]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

def merge_column_b_rows(workbook, n):
    sheet = workbook.active
    max_row = sheet.max_row

    start_row = 2
    while start_row <= max_row:
        end_row = min(start_row + n - 1, max_row)
        if start_row != end_row:
            cell_range = f"B{start_row}:B{end_row}"
            sheet.merge_cells(cell_range)
        start_row += n

In [ ]:
merge_column_b_rows(wb, len(SUPPORTED_MODELS))

In [ ]:
# Save the workbook
output_file = "Model_Responses_New_Models.xlsx"
wb.save(output_file)
print(f"\nExcel spreadsheet saved as {output_file}")